In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e6caf9b2b62e3f90bb350b817b8f64ee1e83bed5549b6056b3b8ea305c1ddb22
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, BooleanType

spark = SparkSession.builder.appName("Crime data ML").getOrCreate()

In [4]:
df = spark.read.csv("Crimes_-_2001_to_Present.csv", header=True, inferSchema=True)


In [5]:
df.show()
df.printSchema()
print(f"Number of rows: {df.count()}")

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11037294|   JA371270|03/18/2015 12:00:...|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK|

In [6]:
df1=df.na.drop()


In [7]:
df1.describe().show()

+-------+--------------------+-----------+--------------------+----------------+------------------+-----------------+--------------------+--------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+-------------------+--------------------+
|summary|                  ID|Case Number|                Date|           Block|              IUCR|     Primary Type|         Description|Location Description|              Beat|         District|             Ward|    Community Area|          FBI Code|      X Coordinate|      Y Coordinate|              Year|          Updated On|           Latitude|          Longitude|            Location|
+-------+--------------------+-----------+--------------------+----------------+------------------+-----------------+--------------------+--------------------+------------------+-----------------+-----------------+--

In [8]:
df1.show()

+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|12045583|   JD226426|05/07/2020 10:24:...| 035XX S INDIANA AVE|0820|               THEFT|      $500 AND UNDER|           AP

In [9]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col


# Convert boolean "Arrest" column to string
df1 = df1.withColumn("Arrest", col("Arrest").cast("string"))
df1 = df1.withColumn("Domestic", col("Domestic").cast("string"))

In [43]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# Columns to apply label encoding
columns_to_encode = ["Primary Type", "Description","Arrest", "Domestic", "FBI Code", "Year"]


# Create a StringIndexer for each column
indexers = [StringIndexer(inputCol=col, outputCol=col+"_encoded", handleInvalid="skip") for col in columns_to_encode]

# Create a pipeline to execute the StringIndexers
pipeline = Pipeline(stages=indexers)

# Fit and transform the data using the pipeline
df2 = pipeline.fit(df1).transform(df1)

# Show the result
df2.show(20)


+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+--------------------+-------------------+--------------+----------------+----------------+------------+
|      ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|Primary Type_encoded|Description_encoded|Arrest_encoded|Domestic_encoded|FBI Code_encoded|Year_encoded|
+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------

In [11]:
df2.printSchema()




root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Primary Type_encoded: double (nullable = false)
 |-- Description_encoded: double (nullable = f

In [12]:
###Random forest


from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("RandomForestExample").getOrCreate()

# Assuming df2 is your DataFrame with label-encoded columns
# If not, replace df2 with your actual DataFrame

# Assemble feature vectors
feature_columns = ["Primary Type_encoded", "Description_encoded", "Arrest_encoded", "Domestic_encoded", "FBI Code_encoded", "Year_encoded"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df2_assembled = assembler.transform(df2)

# Split the data into training and testing sets
(train_data, test_data) = df2_assembled.randomSplit([0.8, 0.2], seed=123)

# Define and train the Random Forest Classifier
rf = RandomForestClassifier(labelCol="Primary Type_encoded", featuresCol="features", numTrees=10, maxBins=510)

model = rf.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model using MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="Primary Type_encoded", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

# Additional metrics: precision, recall, F1 score
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1_score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

# Optional: Print the feature importances
print("Feature Importances:")
for feature, importance in zip(feature_columns, model.featureImportances):
    print(f"{feature}: {importance}")


Accuracy: 0.77088948787062
Precision: 0.7174360349080305
Recall: 0.77088948787062
F1 Score: 0.7288409979993241
Feature Importances:
Primary Type_encoded: 0.5669425430754009
Description_encoded: 0.0781621749507504
Arrest_encoded: 0.0
Domestic_encoded: 0.008836572398426343
FBI Code_encoded: 0.3459305752597785
Year_encoded: 0.00012813431564367164


In [13]:
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Primary Type_encoded: double (nullable = false)
 |-- Description_encoded: double (nullable = f

In [14]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import SparkSession

# Assuming spark is your SparkSession and df2 is your DataFrame
# If not, replace spark and df2 with your actual SparkSession and DataFrame

# Define the fraction of data to sample (e.g., 0.25 for 25%)
fraction_to_sample = 0.25

# Create a subset of the data using the sample method
df2_sampled = df2.sample(fraction=fraction_to_sample, seed=42)

# Create a feature vector
feature_cols = ["Primary Type_encoded", "Description_encoded", "Arrest_encoded", "Domestic_encoded", "FBI Code_encoded", "Year_encoded"]
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="featureslog")
df_assembled = vector_assembler.transform(df2_sampled)

# Assuming label column is "Primary Type_encoded"
lr = LogisticRegression(labelCol="Primary Type_encoded", featuresCol="featureslog")

# Create ParamGrid for Cross Validation
param_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Create a 5-fold CrossValidator
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=param_grid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="Primary Type_encoded", metricName="accuracy"),
                          numFolds=5)

# Run cross-validation, and choose the best set of parameters
cv_model = crossval.fit(df_assembled)

# Get the best model from cross-validation
best_model = cv_model.bestModel

# Display the best parameters
print("Best Parameters:")
print("Regularization Parameter (regParam):", best_model._java_obj.getRegParam())
print("Elastic Net Parameter (elasticNetParam):", best_model._java_obj.getElasticNetParam())

# Make predictions on the test set
predictions = best_model.transform(df_assembled)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="Primary Type_encoded", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy: {:.2%}".format(accuracy))

# Additional metrics if needed
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print("Weighted Precision: {:.2%}".format(precision))
print("Weighted Recall: {:.2%}".format(recall))
print("F1 Score: {:.2%}".format(f1))


Best Parameters:
Regularization Parameter (regParam): 0.01
Elastic Net Parameter (elasticNetParam): 1.0
Accuracy: 64.83%
Weighted Precision: 59.26%
Weighted Recall: 64.83%
F1 Score: 61.41%


In [47]:
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Primary Type_encoded: double (nullable = false)
 |-- Description_encoded: double (nullable = f

In [46]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Assuming you have a label column named 'Primary Type_encoded'
label_col = 'Primary Type_encoded'

# Create a feature vector without explicitly creating an output column
feature_cols = [
    'Description_encoded',
    'Arrest_encoded',
    'Domestic_encoded',
    'FBI Code_encoded',
    'Year_encoded'
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="logisticfeatures21")
lr = LogisticRegression(featuresCol="logisticfeatures21", labelCol=label_col)

# Create a pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Transform the data using the assembler
df2 = assembler.transform(df2)

# Split the data into training and testing sets
(train_data, test_data) = df2.randomSplit([0.8, 0.2], seed=42)

# Train the model
model = pipeline.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

# Print the results
print(f'Accuracy: {accuracy:.4f}')


IllegalArgumentException: ignored

In [38]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LinearSVC
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Assuming you have a label column named 'Primary Type_encoded'
label_col = 'Primary Type_encoded'

# Check for null values
df2.select(*feature_cols, label_col).na.drop().count()

# Create a feature vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="oplinsvc", handleInvalid="skip")
df2 = assembler.transform(df2)

# Split the data into training and testing sets
(train_data, test_data) = df2.randomSplit([0.8, 0.2], seed=42)

# Create a LinearSVC model
lsvc = LinearSVC(maxIter=10, regParam=0.1, labelCol=label_col, featuresCol='oplinsvc')

# Train the model
model = lsvc.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

# Print the results
print(f'Accuracy: {accuracy:.4f}')


Py4JJavaError: ignored

In [23]:
###GBTClassifier

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Assuming you have loaded your dataset into the DataFrame df2

# Assemble features
feature_cols = ['Primary Type_encoded', 'Description_encoded', 'Domestic_encoded', 'FBI Code_encoded', 'Year_encoded']
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features5')

# Convert the 'Arrest' column to a numeric type
label_indexer = StringIndexer(inputCol='Arrest', outputCol='label_arrest')  # Choose a different name

# Create GBT classifier with an increased maxBins value
gbt = GBTClassifier(labelCol='label_arrest', featuresCol='features5', maxBins=512, maxIter=10)  # Adjust maxBins value

# Create pipeline
pipeline = Pipeline(stages=[assembler, label_indexer, gbt])

# Split the data into training and testing sets
train_data, test_data = df2.randomSplit([0.8, 0.2], seed=42)

# Fit the model
model = pipeline.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol='label_arrest', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print(f'Accuracy: {accuracy}')

# Show confusion matrix
conf_matrix = predictions.groupBy('Arrest', 'prediction').count()
conf_matrix.show()


Accuracy: 0.8797101449275362
+------+----------+-----+
|Arrest|prediction|count|
+------+----------+-----+
|  true|       1.0|   48|
|  true|       0.0|   67|
| false|       1.0|   16|
| false|       0.0|  559|
+------+----------+-----+

